In [192]:
from Layers import FullyConnected, Helpers, ReLU, SoftMax, Loss
from Optimization import Optimizers
import copy
import matplotlib.pyplot as plt

In [251]:
class NeuralNetwork:
    def __init__(self, optimizer):
        self.optimizer = optimizer
        self.loss = []
        self.layers = []
        self.data_layer = None
        self.loss_layer = None
        
    def forward(self):
        self.input_tensor, self.label_tensor = self.data_layer.next()
        output = self.input_tensor
        for layer in self.layers:
            output = layer.forward(output)
        return output
    
    def backward(self):
        output = self.loss_layer.forward(self.input_tensor,self.label_tensor)
        for layer in reversed(self.layers):
            output = layer.backward(output)
        
    def append_layer(self, layer):
        if layer.trainable:
            layer.optimizer = copy.deepcopy(self.optimizer)
        self.layers.append(layer)
        
    def train(self, iterations):
        for _ in range(iterations):
            output = self.forward()
            self.backward()
            self.loss.append(self.loss_layer.forward(output,self.label_tensor))
            
    def test(self, input_tensor):
        self.input_tensor = input_tensor
        output = self.forward()
        return output

In [244]:
net = NeuralNetwork(Optimizers.Sgd(1))
categories = 3
input_size = 4
net.data_layer = Helpers.IrisData(50)
net.loss_layer = Loss.CrossEntropyLoss()
fcl_1 = FullyConnected.FullyConnected(input_size, categories)
net.append_layer(fcl_1)
net.append_layer(ReLU.ReLU())
fcl_2 = FullyConnected.FullyConnected(categories, categories)
net.append_layer(fcl_2)
net.append_layer(SoftMax.SoftMax())

out = net.forward()
out2 = net.forward()

c:\Users\ADE17\anaconda3\anaconda2\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [250]:
out2.any()

True

In [243]:
data, labels = net.data_layer.get_test_set()

results = net.test(data)
index_maximum = np.argmax(results, axis=1)
one_hot_vector = np.zeros_like(results)
for i in range(one_hot_vector.shape[0]):
    one_hot_vector[i, index_maximum[i]] = 1

correct = 0.
wrong = 0.
for column_results, column_labels in zip(one_hot_vector, labels):
    if column_results[column_labels > 0].all() > 0:
        correct += 1
    else:
        wrong += 1

accuracy = correct / (correct + wrong)
print('\nOn the Iris dataset, we achieve an accuracy of: ' + str(accuracy * 100) + '%')


On the Iris dataset, we achieve an accuracy of: 34.0%
